In [1]:
# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *

chrip freq 7000, 11000.0
chrip freq 5000, 10000.0


In [50]:
data_len = 100
bin1 = np.random.randint(0, 2, size=data_len)
bin2 = np.random.randint(0, 2, size=data_len)

ENABLE_PAUSE = 0
PAUSE_TIME_RATIO = 0.5
ENABLE_SHIFT_CORR = 0

def fsk_modulation(binary_data, rate, symbol_freq, fsk_freq):
    symbol_samples = int(rate / symbol_freq)
    pause_samples = int(symbol_samples * PAUSE_TIME_RATIO) if ENABLE_PAUSE else 0
    symbol_time = 1 / symbol_freq

    print(f"Min FSK freq: {symbol_freq}")
    assert fsk_freq >= symbol_freq

    t = np.linspace(0, symbol_time, symbol_samples+1)[:-1]
    one = sine_wave(fsk_freq)(t)
    zero = sine_wave(2*fsk_freq)(t)
    one = np.concatenate((one, np.zeros(pause_samples)))
    zero = np.concatenate((zero, np.zeros(pause_samples)))

    fsk_wave = np.concatenate([one if bit == 1 else zero for bit in binary_data])
    return fsk_wave

def fsk_demodulation(expected_bits, receive_wave, rate, symbol_freq, fsk_freq):
    symbol_samples = int(rate / symbol_freq)
    pause_samples = int(symbol_samples * PAUSE_TIME_RATIO) if ENABLE_PAUSE else 0

    symbol_time = 1 / symbol_freq

    t = np.linspace(0, symbol_time, symbol_samples+1)[:-1]
    one = sine_wave(fsk_freq)(t)
    zero = sine_wave(2*fsk_freq)(t)

    receive_fsk_wave = receive_wave

    num_recv = 0
    receive_bits = np.zeros(expected_bits)
    start = 0
    wave_lens = []

    err_cnt = 0
    while num_recv < expected_bits:
        if ENABLE_SHIFT_CORR:
            end = start + int(symbol_samples * 1.1) + pause_samples
        else:
            end = start + int(symbol_samples) + pause_samples

        wave = receive_fsk_wave[start:end]
        wave = np.concatenate((wave, np.zeros(end - start - len(wave))))

        # plot_audio(wave, rate, seconds=None)
        # plot_fft(wave, rate, seconds=None, freq_range=(500, 2500))

        one_corr_arr = np.abs(np.correlate(wave, one, 'full')[:len(wave)])
        zero_corr_arr = np.abs(np.correlate(wave, zero, 'full')[:len(wave)])
        one_corr = np.max(one_corr_arr)
        zero_corr = np.max(zero_corr_arr)
        bit = 1 if one_corr > zero_corr else 0

        if ENABLE_SHIFT_CORR:
            wave_len = (np.argmax(one_corr_arr) if bit == 1 else np.argmax(zero_corr_arr)) + pause_samples
        else:
            wave_len = symbol_samples + pause_samples
        start += wave_len
        wave_lens.append(wave_len)

        if bit != bin1[num_recv]:
            # print(f"idx={num_recv}, bit={bit}, truth={bin1[num_recv]}, wave_len={wave_len} one_corr = {one_corr}, zero_corr = {zero_corr}")
            # if err_cnt == 0:
            #     # plot_audio(wave, rate, seconds=None)
            #     plot_fft(wave, rate, seconds=None)
            err_cnt += 1

        receive_bits[num_recv] = bit
        num_recv += 1
    return receive_bits

rate = 48000

symbol_freq = 1000
fsk_freq = symbol_freq * 2
fsk_wave = fsk_modulation(bin1, rate, symbol_freq, fsk_freq)

record_wave, rate, raw_record_wave = play_and_record_precise2(fsk_wave, rate)
record_wave = -record_wave
# record_wave = fsk_wave

print(len(record_wave), len(fsk_wave))

receive_bits = fsk_demodulation(len(bin1), record_wave.copy(), rate, symbol_freq, fsk_freq)

similarity = np.sum(receive_bits == bin1) / len(bin1)
print(f"Similarity: {similarity}")

transfer_time = len(fsk_wave) / rate
bits = len(bin1)
bps = bits / transfer_time
print(f"Transfer Time: {transfer_time} s, {bps} bps, {bps/1000} kbps")

Min FSK freq: 1000
max corr 0.3144859787933879
max corr 0.24492680009296988
60678 65485
4807 4800
Similarity: 1.0
Transfer Time: 0.1 s, 1000.0 bps, 1.0 kbps


In [50]:
plt.figure()
plt.plot(record_wave)
plt.plot(fsk_wave)
plt.show()

plt.figure()
plt.plot(raw_record_wave)
plt.show()

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5e1ff4b607b0
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5e1ff4ced240
